In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import scipy.stats
import pickle
import copy
import random 
import math
np.set_printoptions(suppress=True)
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import r2_score

In [2]:
# Load change point data
#cp_data = pd.read_csv('d:/iat/change_point_chains.csv')
cp_data = pd.read_csv('c:/Users/User1/Documents/change_point_chains.csv')

In [3]:
cp_data.head()

rt_incongruent_cp1_1  rt_incongruent_cp1_2  rt_incongruent_cp1_3  \
0             18.568622             19.204640             19.441773   
1             18.564692             19.063859             19.449946   
2             18.670911             19.089062             19.424874   
3             18.686892             19.085425             19.420557   
4             18.716680             19.191312             19.396661   

   rt_incongruent_cp2_1  rt_incongruent_cp2_2  rt_incongruent_cp2_3  \
0             61.732553             61.188357             61.779520   
1             61.693823             61.803501             61.822303   
2             61.579319             61.867246             61.666574   
3             61.280671             61.661901             62.164915   
4             61.749466             61.194077             62.048571   

   rt_congruent_cp1_1  rt_congruent_cp1_2  rt_congruent_cp1_3  \
0           17.682671           17.185956           17.404444   
1           17.657749           17.230830           17.414384   
2           17.726489           17.210877           17.415957   
3           17.715387           17.189889           17.418452   
4           17.704557           17.249838           17.396202   

   rt_congruent_cp2_1  ...   a2_cp1_3   a2_cp2_1   a2_cp2_2   a2_cp2_3  \
0           63.057482  ...  17.540687  40.864191  40.199761  39.952542   
1           63.155649  ...  17.573235  41.031973  40.098305  39.489471   
2           62.812527  ...  17.617094  41.116245  40.144171  39.446601   
3           62.933087  ...  17.650977  41.081245  40.079469  39.752541   
4           62.938967  ...  17.642364  41.005270  40.422889  40.281771   

     t_cp1_1    t_cp1_2    t_cp1_3  t_minus_cp1_1  t_minus_cp1_2  \
0  13.854103  15.664327  14.211842      16.530868      13.109360   
1  13.891641  15.640267  14.283838      16.329123      13.605902   
2  13.924335  15.639210  14.264195      16.525654      12.831305   
3  13.747357  15.625085  14.444849      14.955710      13.042342   
4  13.777902  15.589506  14.557520      14.332638      13.950791   

   t_minus_cp1_3  
0      13.745686  
1      14.100602  
2      14.052620  
3      13.327811  
4      13.885100  

[5 rows x 42 columns]

In [4]:
cp_data.columns

Index(['rt_incongruent_cp1_1', 'rt_incongruent_cp1_2', 'rt_incongruent_cp1_3',
       'rt_incongruent_cp2_1', 'rt_incongruent_cp2_2', 'rt_incongruent_cp2_3',
       'rt_congruent_cp1_1', 'rt_congruent_cp1_2', 'rt_congruent_cp1_3',
       'rt_congruent_cp2_1', 'rt_congruent_cp2_2', 'rt_congruent_cp2_3',
       'v1_cp1_1', 'v1_cp1_2', 'v1_cp1_3', 'v1_cp2_1', 'v1_cp2_2', 'v1_cp2_3',
       'v2_cp1_1', 'v2_cp1_2', 'v2_cp1_3', 'v2_cp2_1', 'v2_cp2_2', 'v2_cp2_3',
       'a1_cp1_1', 'a1_cp1_2', 'a1_cp1_3', 'a1_cp2_1', 'a1_cp2_2', 'a1_cp2_3',
       'a2_cp1_1', 'a2_cp1_2', 'a2_cp1_3', 'a2_cp2_1', 'a2_cp2_2', 'a2_cp2_3',
       't_cp1_1', 't_cp1_2', 't_cp1_3', 't_minus_cp1_1', 't_minus_cp1_2',
       't_minus_cp1_3'],
      dtype='object')

In [5]:
#est_data = pd.read_csv('D:/iat/final_iat.csv')
est_data = pd.read_csv('c:/Users/User1/Documents/data.csv')

In [6]:
est_data = est_data.rename(columns={"agex": "age"})
est_data.head()

age  Mn_RT__correct_incongruent  Mn_RT_correct_congruent        v1  \
0   21                    1.016877                 0.814033  1.368002   
1   51                    0.748296                 0.678105  1.736750   
2   21                    0.636271                 0.547907  1.208634   
3   21                    0.891037                 0.630875  1.175065   
4   23                    1.235053                 0.888704  1.368112   

         v2        a1        a2     tplus    tminus  
0  2.444254  2.065458  2.438723  0.318620  1.274719  
1  2.533051  1.258957  1.268246  0.440695  1.018225  
2  2.495442  1.124924  1.064379  0.363593  0.802901  
3  2.375113  1.869280  1.723974  0.259389  2.181388  
4  1.416175  2.293509  1.543404  0.433872  1.259282

In [7]:
est_data.columns

Index(['age', 'Mn_RT__correct_incongruent', 'Mn_RT_correct_congruent', 'v1',
       'v2', 'a1', 'a2', 'tplus', 'tminus'],
      dtype='object')

# Plot with posterior means

In [8]:
def fit_and_plot_ridge(est_data, cp_data, 
                       scale_factors,
                       bottoms,
                       figsize = (22, 16), 
                       which = 'congruent', 
                       color_pred='#f26d6d', 
                       fontsize_metric = 18, 
                       fontsize_lab = 20, 
                       fontsize_legend = 16):

    f, axarr = plt.subplots(2, 2, figsize=figsize)


    y_labels = ['Mean correct RT (seconds)', 'Mental speed', 'Decision caution', 'Non-decision time']

    if which == 'congruent':
        change_points = [(['rt_congruent_cp1_1', 'rt_congruent_cp1_2', 'rt_congruent_cp1_3',], 
                          ['rt_congruent_cp2_1', 'rt_congruent_cp2_2', 'rt_congruent_cp2_3']),
                         (['v2_cp1_1', 'v2_cp1_2', 'v2_cp1_3'], 
                          ['v2_cp2_1', 'v2_cp2_2', 'v2_cp2_3']),
                         (['a2_cp1_1', 'a2_cp1_2', 'a2_cp1_3'], 
                          ['a2_cp2_1', 'a2_cp2_2', 'a2_cp2_3']),
                         (['t_cp1_1', 't_cp1_2', 't_cp1_3'],)]

        predictors = ['Mn_RT_correct_congruent', 'v2', 'a2', 'tminus']
    else:
        change_points = [(['rt_incongruent_cp1_1', 'rt_incongruent_cp1_2', 'rt_incongruent_cp1_3',], 
                          ['rt_incongruent_cp2_1', 'rt_incongruent_cp2_2', 'rt_incongruent_cp2_3']),
                         (['v1_cp1_1', 'v1_cp1_2', 'v1_cp1_3'], 
                          ['v1_cp2_1', 'v1_cp2_2', 'v1_cp2_3']),
                         (['a1_cp1_1', 'a1_cp1_2', 'a1_cp1_3'], 
                          ['a1_cp2_1', 'a1_cp2_2', 'a1_cp2_3']),
                         (['t_minus_cp1_1', 't_minus_cp1_2', 't_minus_cp1_3'],)]

        predictors = ['Mn_RT__correct_incongruent', 'v1', 'a1', 'tplus']

    
    # Drop NA means
    est_data = est_data.dropna(subset=[predictors[0]])
    

    # Group by age for mean age
    est_by_age = est_data.groupby('age', as_index=False).mean()
    est_by_age_std = est_data.groupby('age', as_index=False).std()

    # Extract ages
    age_mean = est_data['age'].values.mean()
    age_std = est_data['age'].values.std() 
    age_pred = est_by_age['age'].values
    age = est_data['age'].values

    for i, (ax, cp_arr, p) in enumerate(zip(axarr.flat, change_points, predictors)):

        est = est_data[p].astype(np.float32)
        est_true = est_by_age[p].astype(np.float32)
        est_true_std = est_by_age_std[p].astype(np.float32)

        cp_means = []
        for cpi, cp in enumerate(cp_arr):

            # Plot change point
            cp_data_h = cp_data[cp].values.flatten()

            # Legend
            if cpi == 0:
                ax.hist(cp_data_h, weights=scale_factors[i]*np.ones_like(cp_data_h),
                        alpha=0.75, color='#8f2727', bins=65, bottom=bottoms[i], label='Change point variability')
            else:
                ax.hist(cp_data_h, weights=scale_factors[i]*np.ones_like(cp_data_h),
                        alpha=0.75, color='#8f2727', bins=65, bottom=bottoms[i])


            # Compute mean of the change points
            cp_mean = cp_data_h.mean()
            if cpi == 0:
                ax.axvline(cp_mean, color='black', linestyle='dashed', label='Change point')
            else:
                ax.axvline(cp_mean, color='black', linestyle='dashed')

            # Add to change point array
            cp_means.append(int(cp_mean))

        # First segment
        idx = age <= cp_means[0]
        x_pred = age_pred[age_pred <= cp_means[0]]

        x_p = age[idx][:, np.newaxis]
        y_p = est[idx]
        model = BayesianRidge(compute_score=True).fit(x_p, y_p)
        y_mean, y_std = model.predict(x_pred[:, np.newaxis], return_std=True)
        print (str(p) + " First segment b = " + str(model.coef_)+ ", 95% HDI"+
                       str(scipy.stats.norm.ppf(.025, model.coef_, math.sqrt(model.sigma_)))+
                       str(scipy.stats.norm.ppf(.975, model.coef_, math.sqrt(model.sigma_))))
        # Plot model predictions
        ax.fill_between(x_pred, y_mean-y_std, y_mean+y_std, color=color_pred, 
                        alpha=0.3)
        ax.plot(x_pred, y_mean, color=color_pred, alpha=0.8, lw=3)

        for ci, cp_mean in enumerate(cp_means):

            # Last segment
            if ci == len(cp_means) - 1:
                idx = age >= cp_mean
                x_pred = age_pred[age_pred >= cp_mean]

                x_p = age[idx][:, np.newaxis]
                y_p = est[idx]
                model = BayesianRidge(compute_score=True).fit(x_p, y_p)
                y_mean, y_std = model.predict(x_pred[:, np.newaxis], return_std=True)
                print (str(p) + " Last segment b = " + str(model.coef_)+ ", 95% HDI"+
                       str(scipy.stats.norm.ppf(.025, model.coef_, math.sqrt(model.sigma_)))+
                       str(scipy.stats.norm.ppf(.975, model.coef_, math.sqrt(model.sigma_))))
                # Plot model predictions
                ax.fill_between(x_pred, y_mean-y_std, y_mean+y_std, color=color_pred, 
                                alpha=0.3, label=r'Predictive variability ($\pm\,1\,SD$)')
                ax.plot(x_pred, y_mean, color=color_pred, alpha=0.8, lw=3, label='Predictive mean')
            # Middle segments
            else:
                idx = (age >= cp_means[ci]) & (age <= cp_means[ci+1])
                x_pred = age_pred[(age_pred >= cp_means[ci]) & (age_pred <= cp_means[ci+1])]

                x_p = age[idx][:, np.newaxis]
                y_p = est[idx]
                model = BayesianRidge(compute_score=True).fit(x_p, y_p)
                y_mean, y_std = model.predict(x_pred[:, np.newaxis], return_std=True)
                print (str(p) + " Middle segment b = " + str(model.coef_)+", 95% HDI"+
                       str(scipy.stats.norm.ppf(.025, model.coef_, math.sqrt(model.sigma_)))+
                       str(scipy.stats.norm.ppf(.975, model.coef_, math.sqrt(model.sigma_))))
                # Plot model predictions
                ax.fill_between(x_pred, y_mean-y_std, y_mean+y_std, color=color_pred, 
                                alpha=0.3)
                ax.plot(x_pred, y_mean, color=color_pred, alpha=0.8, lw=3)
        ax.scatter(age_pred, est_true, color='black', s=45, label='Parameter mean')

        ax.errorbar(age_pred[::2], est_true[::2], est_true_std[::2], capsize=4,
                    color='black', alpha=0.35, linewidth=1, label=r'Parameter variability ($\pm\,1\,SD$)')
        sns.despine(ax = ax)
        ax.set_ylabel(y_labels[i], fontsize=fontsize_lab)
        ax.set_xlabel('Age', fontsize=fontsize_lab)
        ax.xaxis.set_tick_params(labelsize=fontsize_lab-2)
        ax.yaxis.set_tick_params(labelsize=fontsize_lab-2)
        if i == 1:
            loc = 'lower center'
        else:
            loc = 'upper center'
        ax.grid(alpha=0.2)
        if i == 0:
            
            handles, labels = ax.get_legend_handles_labels()
            order = [4, 5, 0, 2, 1, 3]
            lg = ax.legend([handles[il] for il in order],[labels[il] for il in order], 
                           loc='upper center', ncol=1, fontsize=fontsize_legend, bbox_to_anchor=(0.45, 1.0))

    f.tight_layout()
    f.savefig('figures/model_preds_{}.pdf'.format(which), 
                dpi=300, 
                format='pdf', 
                bbox_inches='tight')

In [9]:
scale_factors = [0.000005, 0.0000075, 0.000011, 0.000004]
bottoms = [0.45, 0.5, 1.0, 0.2]
fit_and_plot_ridge(est_data, cp_data, scale_factors, bottoms, which='incongruent')

Mn_RT__correct_incongruent First segment b = [-0.02425402], 95% HDI[-0.02505244][-0.02345561]
Mn_RT__correct_incongruent Middle segment b = [0.00567443], 95% HDI[0.00562163][0.00572723]
Mn_RT__correct_incongruent Last segment b = [0.02204741], 95% HDI[0.02074385][0.02335098]
v1 First segment b = [0.033786], 95% HDI[0.03321322][0.03435877]
v1 Middle segment b = [-0.00040083], 95% HDI[-0.00057921][-0.00022245]
v1 Last segment b = [-0.01959454], 95% HDI[-0.02084696][-0.01834212]
a1 First segment b = [-0.02467901], 95% HDI[-0.02647958][-0.02287845]
a1 Middle segment b = [0.01084078], 95% HDI[0.01075877][0.0109228]
a1 Last segment b = [0.02061045], 95% HDI[0.01831512][0.02290578]
tplus First segment b = [-0.00645536], 95% HDI[-0.00760776][-0.00530297]
tplus Last segment b = [0.00209757], 95% HDI[0.00208749][0.00210764]


In [10]:
scale_factors = [0.000005, 0.000007, 0.000003, 0.000006]
bottoms = [0.3, 0.5, 1.0, 0.01]
fit_and_plot_ridge(est_data, cp_data, scale_factors, bottoms, which='congruent')

Mn_RT_correct_congruent First segment b = [-0.02690211], 95% HDI[-0.02779415][-0.02601008]
Mn_RT_correct_congruent Middle segment b = [0.00485672], 95% HDI[0.00481679][0.00489666]
Mn_RT_correct_congruent Last segment b = [0.01040364], 95% HDI[0.00945101][0.01135626]
v2 First segment b = [0.06505983], 95% HDI[0.06397655][0.06614312]
v2 Middle segment b = [0.00533151], 95% HDI[0.00511959][0.00554342]
v2 Last segment b = [-0.01528134], 95% HDI[-0.01665822][-0.01390445]
a2 First segment b = [-0.02599653], 95% HDI[-0.027819][-0.02417405]
a2 Middle segment b = [0.01555766], 95% HDI[0.01538922][0.01572609]
a2 Last segment b = [0.00471658], 95% HDI[0.00442326][0.00500989]
tminus First segment b = [-0.01986997], 95% HDI[-0.02988804][-0.00985189]
tminus Last segment b = [0.02029458], 95% HDI[0.02017643][0.02041273]
